In [1]:
'CIFAR10 '

'CIFAR10 '

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import os
import pickle
import numpy as np

Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [3]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
y_train

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.]])

In [6]:
model = Sequential()

In [7]:
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [8]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [9]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [10]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [10]:
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
 3680/50000 [=>............................] - ETA: 76s - loss: 2.2887 - acc: 0.1277

KeyboardInterrupt: 

In [11]:
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

In [12]:
    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [13]:
    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

Epoch 1/10
1562/1562 [==============================] - 76s - loss: 1.9922 - acc: 0.2592 - val_loss: 2.3091 - val_acc: 0.1074
Epoch 2/10
1562/1562 [==============================] - 74s - loss: 1.8456 - acc: 0.3216 - val_loss: 2.3058 - val_acc: 0.1275
Epoch 3/10
1562/1562 [==============================] - 74s - loss: 1.7832 - acc: 0.3512 - val_loss: 2.3058 - val_acc: 0.1027
Epoch 4/10
1562/1562 [==============================] - 74s - loss: 1.7398 - acc: 0.3682 - val_loss: 2.3089 - val_acc: 0.1041
Epoch 5/10
1562/1562 [==============================] - 73s - loss: 1.7060 - acc: 0.3838 - val_loss: 2.3182 - val_acc: 0.1149
Epoch 6/10
1562/1562 [==============================] - 73s - loss: 1.6782 - acc: 0.3941 - val_loss: 2.3336 - val_acc: 0.1090
Epoch 7/10
1562/1562 [==============================] - 73s - loss: 1.6602 - acc: 0.4005 - val_loss: 2.3342 - val_acc: 0.1095
Epoch 8/10
1562/1562 [==============================] - 73s - loss: 1.6399 - acc: 0.4081 - val_loss: 2.3577 - val_acc:

In [16]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at /gpfs/global_fs01/sym_shared/YPProdSpark/user/s45f-65f67acefe15fa-238764f4c881/notebook/work/saved_models/keras_cifar10_trained_model.h5 


In [17]:
# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


In [18]:
keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

In [19]:
# Evaluate model with test data set and share sample prediction results
evaluation = model.evaluate_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

Model Accuracy = 0.74


In [20]:
predict_gen = model.predict_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)

In [21]:
for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

Actual Label = cat vs. Predicted Label = frog
Actual Label = ship vs. Predicted Label = bird
Actual Label = ship vs. Predicted Label = ship
Actual Label = airplane vs. Predicted Label = airplane
Actual Label = frog vs. Predicted Label = cat
Actual Label = frog vs. Predicted Label = horse
Actual Label = automobile vs. Predicted Label = bird
Actual Label = frog vs. Predicted Label = frog
Actual Label = cat vs. Predicted Label = ship
Actual Label = automobile vs. Predicted Label = ship
Actual Label = airplane vs. Predicted Label = airplane
Actual Label = truck vs. Predicted Label = bird
Actual Label = dog vs. Predicted Label = truck
Actual Label = horse vs. Predicted Label = cat
Actual Label = truck vs. Predicted Label = dog
Actual Label = ship vs. Predicted Label = ship
Actual Label = dog vs. Predicted Label = ship
Actual Label = horse vs. Predicted Label = automobile
Actual Label = ship vs. Predicted Label = automobile
Actual Label = frog vs. Predicted Label = horse
Actual Label = horse